In [4]:
# Recipe Recommendation System
# This system recommends recipes based on user-provided ingredients from your existing dataset

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import ast
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Load your existing dataset
def load_data(file_path):
    """
    Load your recipes.csv dataset
    """
    try:
        df = pd.read_csv(file_path)
        print(f"✅ Dataset loaded successfully with {len(df)} recipes")
        print(f"📊 Columns available: {list(df.columns)}")
        
        # Show basic info about the dataset
        print(f"\n📋 First few recipe names:")
        for i, name in enumerate(df['recipe_name'].head().values):
            print(f"   {i+1}. {name}")
            
        return clean_dataframe(df)
        
    except FileNotFoundError:
        print(f"❌ Error: File '{file_path}' not found.")
        print("💡 Please make sure 'recipes.csv' is in the same directory as this notebook")
        return None
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None

def clean_dataframe(df):
    """
    Clean the dataframe and convert columns to proper data types
    """
    df_clean = df.copy()
    
    # Display column information
    print(f"\n🧹 Data cleaning in progress...")
    print(f"   Original shape: {df_clean.shape}")
    
    # Convert numeric columns, handling errors and missing values
    numeric_columns = ['prep_time', 'cook_time', 'total_time', 'servings', 'rating']
    
    for col in numeric_columns:
        if col in df_clean.columns:
            print(f"   Processing {col}...")
            df_clean[col] = df_clean[col].apply(extract_number)
    
    # Handle missing values
    text_columns = ['ingredients', 'directions', 'recipe_name', 'url', 'cuisine_path', 'nutrition', 'timing', 'img_src']
    for col in text_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna('')
    
    # Fill NaN numeric values with 0
    for col in numeric_columns:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].fillna(0)
    
    print(f"   Final shape: {df_clean.shape}")
    print("✅ Data cleaning completed successfully")
    
    return df_clean

def extract_number(value):
    """
    Extract numeric value from mixed data types
    """
    if pd.isna(value) or value == '':
        return 0
    
    # If already numeric, return as is
    if isinstance(value, (int, float)):
        return float(value)
    
    # If string, try to extract number
    if isinstance(value, str):
        # Remove text and keep numbers
        numbers = re.findall(r'\d+\.?\d*', value)
        if numbers:
            return float(numbers[0])
        # Handle time strings like "30 mins", "1 hour"
        time_mapping = {
            'hour': 60, 'hours': 60, 'hr': 60, 'hrs': 60,
            'minute': 1, 'minutes': 1, 'min': 1, 'mins': 1
        }
        for unit, multiplier in time_mapping.items():
            if unit in value.lower():
                numbers = re.findall(r'\d+\.?\d*', value)
                if numbers:
                    return float(numbers[0]) * multiplier
    
    return 0

class RecipeRecommender:
    def __init__(self, df):
        """
        Initialize the recommender with recipe data
        """
        self.df = df.copy()
        self.preprocess_data()
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
        self.ingredient_matrix = self._create_ingredient_matrix()
    
    def preprocess_data(self):
        """
        Preprocess the recipe data
        """
        print("🔧 Preprocessing data...")
        self.df['cleaned_ingredients'] = self.df['ingredients'].apply(self._clean_ingredients)
        print("✅ Data preprocessing completed")
    
    def _clean_ingredients(self, ingredients_text):
        """
        Clean and normalize ingredients text
        """
        if pd.isna(ingredients_text) or ingredients_text == '':
            return ""
        
        # Convert to string if not already
        ingredients_str = str(ingredients_text)
        
        try:
            # Try to parse as list if it's in string list format
            if ingredients_str.startswith('[') and ingredients_str.endswith(']'):
                ingredients_list = ast.literal_eval(ingredients_str)
            else:
                # Split by comma or other separators
                ingredients_list = re.split(r'[,|]', ingredients_str)
            
            # Clean each ingredient
            cleaned_ingredients = []
            for ingredient in ingredients_list:
                if isinstance(ingredient, str):
                    # Remove measurements, numbers, and special characters
                    cleaned = re.sub(r'\d+/\d+|\d+\.\d+|\d+', '', ingredient)
                    cleaned = re.sub(r'[^\w\s]', ' ', cleaned)
                    cleaned = ' '.join(cleaned.split()).strip().lower()
                    if cleaned and len(cleaned) > 1:
                        cleaned_ingredients.append(cleaned)
            
            return ' '.join(cleaned_ingredients)
        
        except:
            # Fallback: simple cleaning
            cleaned = re.sub(r'\d+/\d+|\d+\.\d+|\d+', '', ingredients_str)
            cleaned = re.sub(r'[^\w\s]', ' ', cleaned)
            return ' '.join(cleaned.split()).lower()
    
    def _create_ingredient_matrix(self):
        """
        Create TF-IDF matrix for ingredient matching
        """
        print("🔧 Creating ingredient similarity matrix...")
        ingredient_corpus = self.df['cleaned_ingredients'].tolist()
        matrix = self.vectorizer.fit_transform(ingredient_corpus)
        print("✅ Ingredient matrix created")
        return matrix
    
    def find_recipes_by_ingredients(self, user_ingredients, top_n=10, min_similarity=0.1):
        """
        Find recipes that match user-provided ingredients
        """
        # Clean user input
        user_ingredients_clean = self._clean_ingredients(user_ingredients)
        
        if not user_ingredients_clean:
            print("⚠️ No valid ingredients detected.")
            return pd.DataFrame()
        
        print(f"🔍 Searching for recipes with: {user_ingredients_clean}")
        
        # Transform user input to TF-IDF
        user_vector = self.vectorizer.transform([user_ingredients_clean])
        
        # Calculate cosine similarity
        similarities = cosine_similarity(user_vector, self.ingredient_matrix).flatten()
        
        # Get top matching recipes
        similar_indices = similarities.argsort()[::-1]
        matching_indices = [idx for idx in similar_indices if similarities[idx] >= min_similarity][:top_n]
        
        if not matching_indices:
            print("❌ No recipes found matching your ingredients. Try different ingredients.")
            return pd.DataFrame()
        
        # Prepare results
        results = self.df.iloc[matching_indices].copy()
        results['similarity_score'] = similarities[matching_indices]
        
        print(f"✅ Found {len(results)} matching recipes")
        return self._format_results(results)
    
    def _format_results(self, results):
        """
        Format the results with all required columns
        """
        required_columns = [
            'recipe_name', 'prep_time', 'cook_time', 'total_time', 
            'servings', 'ingredients', 'directions', 'rating', 'url', 
            'cuisine_path', 'nutrition', 'timing', 'img_src', 'similarity_score'
        ]
        
        # Ensure all required columns exist
        for col in required_columns:
            if col not in results.columns:
                results[col] = ''
        
        return results[required_columns]
    
    def display_recommendations(self, recommendations):
        """
        Display recipe recommendations in a user-friendly format
        """
        if recommendations.empty:
            print("❌ No recipes found.")
            return
        
        print(f"\n🎯 Found {len(recommendations)} matching recipes:\n")
        print("="*100)
        
        for idx, (_, recipe) in enumerate(recommendations.iterrows(), 1):
            print(f"\n{idx}. {recipe['recipe_name']} ⭐ {recipe['rating']}/5")
            print(f"   📊 Match Score: {recipe['similarity_score']:.3f}")
            print(f"   ⏱️  Prep: {recipe['prep_time']} min | Cook: {recipe['cook_time']} min | Total: {recipe['total_time']} min")
            print(f"   👥 Servings: {recipe['servings']}")
            
            if 'cuisine_path' in recipe and pd.notna(recipe['cuisine_path']) and recipe['cuisine_path'] != '':
                print(f"   🍽️  Cuisine: {recipe['cuisine_path']}")
            
            if 'img_src' in recipe and pd.notna(recipe['img_src']) and recipe['img_src'] != '':
                print(f"   🖼️  Image: {recipe['img_src']}")
            
            if 'url' in recipe and pd.notna(recipe['url']) and recipe['url'] != '':
                print(f"   🔗 Recipe URL: {recipe['url']}")
            
            # Display ingredients preview
            ingredients_text = str(recipe['ingredients'])
            ingredients_preview = ingredients_text[:150] + "..." if len(ingredients_text) > 150 else ingredients_text
            print(f"   🛒 Ingredients: {ingredients_preview}")
            
            print("-" * 80)

def show_recipe_details(recipe):
    """
    Display detailed information for a specific recipe
    """
    print("\n" + "="*80)
    print(f"📖 RECIPE DETAILS: {recipe['recipe_name']}")
    print("="*80)
    
    print(f"\n⭐ Rating: {recipe['rating']}/5")
    print(f"⏱️  Timing: Prep {recipe['prep_time']} min | Cook {recipe['cook_time']} min | Total {recipe['total_time']} min")
    print(f"👥 Servings: {recipe['servings']}")
    
    if 'cuisine_path' in recipe and pd.notna(recipe['cuisine_path']) and recipe['cuisine_path'] != '':
        print(f"🍽️  Cuisine: {recipe['cuisine_path']}")
    
    if 'img_src' in recipe and pd.notna(recipe['img_src']) and recipe['img_src'] != '':
        print(f"🖼️  Image: {recipe['img_src']}")
    
    if 'url' in recipe and pd.notna(recipe['url']) and recipe['url'] != '':
        print(f"🔗 URL: {recipe['url']}")
    
    print(f"\n🛒 INGREDIENTS:")
    ingredients = recipe['ingredients']
    if pd.notna(ingredients) and ingredients != '':
        if isinstance(ingredients, str) and ingredients.startswith('['):
            try:
                ingredients_list = ast.literal_eval(ingredients)
                for i, ingredient in enumerate(ingredients_list, 1):
                    print(f"   {i}. {ingredient}")
            except:
                print(f"   {ingredients}")
        else:
            print(f"   {ingredients}")
    else:
        print("   No ingredients listed")
    
    print(f"\n👩‍🍳 DIRECTIONS:")
    directions = recipe['directions']
    if pd.notna(directions) and directions != '':
        if isinstance(directions, str) and directions.startswith('['):
            try:
                directions_list = ast.literal_eval(directions)
                for i, step in enumerate(directions_list, 1):
                    print(f"   {i}. {step}")
            except:
                print(f"   {directions}")
        else:
            print(f"   {directions}")
    else:
        print("   No directions available")
    
    print(f"\n🥗 NUTRITION INFO:")
    nutrition = recipe['nutrition']
    if pd.notna(nutrition) and nutrition != '' and nutrition != '{}':
        print(f"   {nutrition}")
    else:
        print("   No nutrition information available")
    
    print(f"\n⏰ TIMING DETAILS:")
    timing = recipe['timing']
    if pd.notna(timing) and timing != '' and timing != '{}':
        print(f"   {timing}")
    else:
        print("   No detailed timing information available")

def advanced_analysis(df):
    """
    Perform advanced analysis on the recipe dataset
    """
    print("\n" + "="*50)
    print("📊 DATABASE ANALYSIS")
    print("="*50)
    
    # Basic statistics
    print(f"📈 Total recipes: {len(df):,}")
    print(f"⭐ Average rating: {df['rating'].mean():.2f}/5")
    print(f"⏱️  Average prep time: {df['prep_time'].mean():.1f} min")
    print(f"🍳 Average cook time: {df['cook_time'].mean():.1f} min")
    print(f"⏰ Average total time: {df['total_time'].mean():.1f} min")
    print(f"👥 Average servings: {df['servings'].mean():.1f}")
    
    # Rating distribution
    print(f"\n📊 Rating Distribution:")
    rating_counts = df['rating'].value_counts().sort_index(ascending=False)
    for rating, count in rating_counts.head().items():
        print(f"   ⭐ {rating}: {count} recipes")
    
    # Top cuisines
    if 'cuisine_path' in df.columns:
        print(f"\n🍽️  Top Cuisine Categories:")
        cuisine_counts = df['cuisine_path'].value_counts().head(8)
        for cuisine, count in cuisine_counts.items():
            print(f"   📁 {cuisine}: {count} recipes")

def handle_recipe_details(recommendations):
    """
    Handle the recipe details viewing with proper loop control
    """
    while True:
        print("\nOptions:")
        print("  • Enter recipe number (1, 2, 3, etc.) to view details")
        print("  • Enter 'back' or 'b' to return to search")
        print("  • Enter 'quit' or 'q' to exit the program")
        
        user_choice = input("\n🔍 Your choice: ").strip().lower()
        
        if user_choice in ['back', 'b']:
            return 'back'
        elif user_choice in ['quit', 'q', 'exit']:
            return 'quit'
        else:
            try:
                recipe_num = int(user_choice) - 1
                if 0 <= recipe_num < len(recommendations):
                    show_recipe_details(recommendations.iloc[recipe_num])
                    # After showing details, ask if they want to see another or go back
                    while True:
                        next_action = input("\nEnter 'back' to return to options or 'quit' to exit: ").strip().lower()
                        if next_action in ['back', 'b']:
                            break
                        elif next_action in ['quit', 'q', 'exit']:
                            return 'quit'
                        else:
                            print("⚠️ Please enter 'back' or 'quit'")
                else:
                    print(f"⚠️ Please enter a number between 1 and {len(recommendations)}")
            except ValueError:
                print("⚠️ Please enter a valid number, 'back', or 'quit'")

def main():
    """
    Main function to run the recipe recommendation system
    """
    print("🍳 Recipe Recommendation System")
    print("="*50)
    
    # Load your dataset
    df = load_data('recipes.csv')
    
    if df is None:
        print("❌ Cannot continue without the dataset. Please check if 'recipes.csv' exists.")
        return
    
    # Show analysis of your dataset
    advanced_analysis(df)
    
    # Initialize recommender with your data
    print("\n" + "="*50)
    print("🚀 Initializing Recipe Recommender...")
    recommender = RecipeRecommender(df)
    
    print("\n" + "="*50)
    print("✅ System Ready! You can now search for recipes.")
    
    # Interactive search loop
    while True:
        print("\n" + "="*50)
        print("Enter ingredients you have (separated by commas)")
        print("Examples: 'apples, cinnamon, sugar' or 'chicken, rice, vegetables'")
        print("Type 'quit' to exit the program")
        
        user_input = input("\n🧂 Your ingredients: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("👋 Happy cooking! Goodbye!")
            break
        
        if not user_input:
            print("⚠️ Please enter at least one ingredient.")
            continue
        
        # Get recommendations from your dataset
        print("🔍 Searching your recipe database...")
        recommendations = recommender.find_recipes_by_ingredients(user_input, top_n=10)
        
        # Display results from your dataset
        recommender.display_recommendations(recommendations)
        
        # Option to see more details - with proper loop control
        if not recommendations.empty:
            result = handle_recipe_details(recommendations)
            if result == 'quit':
                print("👋 Happy cooking! Goodbye!")
                break
            # If result is 'back', we just continue to the next search iteration

# Run the system
if __name__ == "__main__":
    main()

🍳 Recipe Recommendation System
✅ Dataset loaded successfully with 1090 recipes
📊 Columns available: ['Unnamed: 0', 'recipe_name', 'prep_time', 'cook_time', 'total_time', 'servings', 'yield', 'ingredients', 'directions', 'rating', 'url', 'cuisine_path', 'nutrition', 'timing', 'img_src']

📋 First few recipe names:
   1. Apple-Cranberry Crostada
   2. Apple Pie by Grandma Ople
   3. Sarah's Homemade Applesauce
   4. Apple Crisp
   5. Apple Pie Filling

🧹 Data cleaning in progress...
   Original shape: (1090, 15)
   Processing prep_time...
   Processing cook_time...
   Processing total_time...
   Processing servings...
   Processing rating...
   Final shape: (1090, 15)
✅ Data cleaning completed successfully

📊 DATABASE ANALYSIS
📈 Total recipes: 1,090
⭐ Average rating: 4.53/5
⏱️  Average prep time: 15.8 min
🍳 Average cook time: 14.5 min
⏰ Average total time: 14.6 min
👥 Average servings: 13.8

📊 Rating Distribution:
   ⭐ 5.0: 83 recipes
   ⭐ 4.9: 52 recipes
   ⭐ 4.8: 133 recipes
   ⭐ 4.7: 19


🧂 Your ingredients:  chicken,rice,vegetables


🔍 Searching your recipe database...
🔍 Searching for recipes with: chicken rice vegetables
✅ Found 10 matching recipes

🎯 Found 10 matching recipes:


1. Greek Lemon Chicken Soup ⭐ 4.5/5
   📊 Match Score: 0.414
   ⏱️  Prep: 25.0 min | Cook: 40.0 min | Total: 1.0 min
   👥 Servings: 16.0
   🍽️  Cuisine: /Soups, Stews and Chili Recipes/Soup Recipes/Chicken Soup Recipes/Chicken and Rice Soup Recipes/
   🖼️  Image: https://www.allrecipes.com/thmb/0YaIza9FtHUXDocyKM1wm90ekMY=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/5795826-50a4cf24ce38481f9190daa45a1e8ec3.jpg
   🔗 Recipe URL: https://www.allrecipes.com/recipe/12963/greek-lemon-chicken-soup/
   🛒 Ingredients: 8 cups chicken broth, ½ cup fresh lemon juice, ½ cup shredded carrots, ½ cup chopped onion, ½ cup chopped celery, 6 tablespoons chicken soup base, ¼ t...
--------------------------------------------------------------------------------

2. Cucumber and Avocado Sushi ⭐ 4.3/5
   📊 Match Score: 0.384
   ⏱️  Prep: 35.0 min | 


🔍 Your choice:  10



📖 RECIPE DETAILS: Coconut Jasmine Rice

⭐ Rating: 4.6/5
⏱️  Timing: Prep 5.0 min | Cook 25.0 min | Total 30.0 min
👥 Servings: 6.0
🍽️  Cuisine: /Side Dish/Rice Side Dish Recipes/
🖼️  Image: https://www.allrecipes.com/thmb/zLaCSeQn5T8Vc4TB_9ZK8eWTSlM=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/8284095-coconut-jasmine-rice-Pasquale-4x3-1-c89e1cef73f3469c862b96bf09ee31aa.jpg
🔗 URL: https://www.allrecipes.com/recipe/238184/coconut-jasmine-rice/

🛒 INGREDIENTS:
   1 (14 ounce) can coconut milk, 1 ¼ cups water, 1 teaspoon white sugar, 1 pinch salt, 1 ½ cups jasmine rice

👩‍🍳 DIRECTIONS:
   Stir coconut milk, water, sugar, and salt together in a saucepan over medium heat until sugar dissolves.
Add rice and bring to a boil. Cover the saucepan, reduce the heat to low, and simmer until rice is tender and liquid is absorbed, 18 to 20 minutes.

🥗 NUTRITION INFO:
   Total Fat 14g 18%, Saturated Fat 12g 62%, Sodium 10mg 0%, Total Carbohydrate 41g 15%, Dietary Fiber 1g 5%, Total Sugars


Enter 'back' to return to options or 'quit' to exit:  quit


👋 Happy cooking! Goodbye!
